In [2]:
from collections import defaultdict, Counter
import re
import pandas as pd

In [3]:
def analyze_game_stats_from_csv(csv_file):
    # Загружаем CSV как текст
    with open(csv_file, 'r') as file:
        lines = file.readlines()

    # Инициализация структуры для хранения статистики
    player_stats = defaultdict(lambda: {
        "total_serves": 0,
        "won_serves": 0,
        "lost_serves": 0,
        "zone_count": Counter(),
        "serve_style": {"P": 0, "F": 0},
        "reception_quality": {"R#": 0, "R+": 0, "R!": 0, "R-": 0, "R$": 0},
        "total_receptions": 0,
        "reception_errors": 0,
        "good_reception_rate": 0.0,
        "best_reception_rate": 0.0,
        "total_attacks": 0,
        "won_attacks": 0,
        "attack_errors": 0,
        "attack_type_F": {"total": 0, "success": 0},
        "attack_success_rate_F": 0.0,
        "total_blocks": 0,
        "total_errors": 0,
        "total_points": 0,
        "total_points": 0,
        "total_errors": 0,
        "result": 0
    })

    # Регулярные выражения для парсинга шифров
    serve_pattern = r"S(\d{1,2})z(\d)([FP])([#-]?)"
    reception_pattern = r"R([#+!--$])n(\d{1,2})"
    attack_pattern = r"A(\d{1,2})([-+OTE]?)z(\d)([FD]?)([#-]?)"
    block_pattern = r"B(\d{1,2})"
    error_pattern = r"E(\d{1,2})([a-zA-Z]*)"

    for line in lines:
        events = line.strip().split(',')  # Удаляем пробелы и разделяем по запятой

        for entry in events:
            # Проверка, является ли шифр подачей
            serve_match = re.match(serve_pattern, entry)
            if serve_match:
                player_num = int(serve_match.group(1))
                zone = int(serve_match.group(2))
                style = serve_match.group(3)
                result = serve_match.group(4)

                # Обновление статистики подачи
                player_stats[player_num]["total_serves"] += 1
                player_stats[player_num]["zone_count"][zone] += 1
                player_stats[player_num]["serve_style"][style] += 1

                if result == '#':
                    player_stats[player_num]["won_serves"] += 1
                elif result == '-':
                    player_stats[player_num]["lost_serves"] += 1
                continue
            
            # Проверка, является ли шифр приемом
            reception_match = re.match(reception_pattern, entry)
            if reception_match:
                quality = reception_match.group(1)
                player_num = int(reception_match.group(2))

                # Обновление статистики приема
                player_stats[player_num]["total_receptions"] += 1
                if quality == "$":
                    player_stats[player_num]["reception_errors"] += 1

                if f"R{quality}" in player_stats[player_num]["reception_quality"]:
                    player_stats[player_num]["reception_quality"][f"R{quality}"] += 1
                continue

            # Проверка, является ли шифр атакой
            attack_match = re.match(attack_pattern, entry)
            if attack_match:
                player_num = int(attack_match.group(1))
                block_touch = attack_match.group(2)
                zone = int(attack_match.group(3))
                attack_type = attack_match.group(4)
                quality = attack_match.group(5)

                # Обновление статистики атаки
                player_stats[player_num]["total_attacks"] += 1
                if block_touch == 'O':
                    player_stats[player_num]["won_attacks"] += 1
                    player_stats[player_num]["attack_type_F"]["total"] += 1

                if block_touch == 'E':
                    player_stats[player_num]["attack_errors"] += 1

                if quality == '#':
                    player_stats[player_num]["won_attacks"] += 1
                elif quality == '-':
                    player_stats[player_num]["attack_errors"] += 1

                if attack_type == 'F':
                    player_stats[player_num]["attack_type_F"]["total"] += 1
                    if quality == '#':
                        player_stats[player_num]["attack_type_F"]["success"] += 1
                continue

            # Проверка, является ли шифр блоком
            block_match = re.match(block_pattern, entry)
            if block_match:
                player_num = int(block_match.group(1))
                player_stats[player_num]["total_blocks"] += 1
                continue
            
            # Проверка, является ли шифр ошибкой
            error_match = re.match(error_pattern, entry)
            if error_match:
                player_num = int(error_match.group(1))
                player_stats[player_num]["total_errors"] += 1
                continue

            print(f"Некорректный формат шифра: {entry}")

    # Вычисление дополнительных метрик для приема и атаки
    for player, stats in player_stats.items():
        total_receptions = stats["total_receptions"]
        if total_receptions > 0:
            good_receptions = stats["reception_quality"]["R#"] + stats["reception_quality"]["R+"]
            stats["good_reception_rate"] = good_receptions / total_receptions
            stats["best_reception_rate"] = stats["reception_quality"]["R#"] / total_receptions

        total_F_attacks = stats["attack_type_F"]["total"]
        if total_F_attacks > 0:
            stats["attack_success_rate_F"] = stats["attack_type_F"]["success"] / total_F_attacks
        
        stats["total_points"]= stats["won_attacks"] + stats["won_serves"] + stats["total_blocks"]
        stats["total_errors"] = stats["attack_errors"] + stats["lost_serves"] + stats["total_errors"] + stats["reception_errors"]
        
        stats["result_player"] = stats["total_points"] - stats["total_errors"]


    # Преобразуем статистику в DataFrame
    data_records = []
    for player, stats in player_stats.items():
        most_common_zone = stats["zone_count"].most_common(1)
        most_common_zone = most_common_zone[0][0] if most_common_zone else None
        data_records.append({
            "Player": player,
            "Total Serves": stats["total_serves"],
            "Won Serves": stats["won_serves"],
            "Lost Serves": stats["lost_serves"],
            "Most Common Zone": most_common_zone,
            "Serve Style P": stats["serve_style"]["P"],
            "Serve Style F": stats["serve_style"]["F"],
            "Total Receptions": stats["total_receptions"],
            "Reception Errors (R--)": stats["reception_errors"],
            "Good Reception Rate (R#+R+)": round(stats["good_reception_rate"], 2),
            "Best Reception Rate (R#)": round(stats["best_reception_rate"], 2),
            "Total Attacks": stats["total_attacks"],
            "Won Attacks": stats["won_attacks"],
            "Attack Errors": stats["attack_errors"],
            "Attack Success Rate F": round(stats["attack_success_rate_F"], 2),
            "Total Blocks": stats["total_blocks"],
            "Total Errors": stats["total_errors"],
            "Total Points": stats["total_points"],
            "Player Result": stats["result_player"] 

        })
    
    # Создаём DataFrame
    df = pd.DataFrame(data_records)
    return df

In [4]:
csv_file_path = r'dev.csv'  # Замените на путь к вашему CSV файлу
df_stats = analyze_game_stats_from_csv(csv_file_path)
df_stats.sort_values(by='Total Attacks', ascending=False)

C:\Users\Ivan\AppData\Local\Temp\ipykernel_14572\3623077965.py:62: FutureWarning: Possible set difference at position 6
  reception_match = re.match(reception_pattern, entry)


,Player,Total Serves,Won Serves,Lost Serves,Most Common Zone,Serve Style P,Serve Style F,Total Receptions,Reception Errors (R--),Good Reception Rate (R#+R+),Best Reception Rate (R#),Total Attacks,Won Attacks,Attack Errors,Attack Success Rate F,Total Blocks,Total Errors,Total Points,Player Result
1,9,2,0,1,5.0,2,0,3,0,1.00,0.67,5,4,0,0.75,0,1,4,3
3,15,6,2,0,6.0,1,5,0,0,0.00,0.00,5,2,1,0.50,0,1,4,3
6,7,1,0,0,1.0,1,0,0,0,0.00,0.00,4,2,1,0.67,1,1,3,2
2,13,3,0,0,6.0,3,0,0,0,0.00,0.00,3,2,1,0.67,0,1,2,1
0,5,8,1,0,6.0,8,0,0,0,0.00,0.00,1,1,0,0.00,0,0,2,2
4,10,0,0,0,NaN,0,0,1,1,0.00,0.00,1,1,0,0.00,0,2,1,-1
7,4,4,0,0,6.0,4,0,0,0,0.00,0.00,1,0,0,0.00,0,0,0,0
5,8,0,0,0,NaN,0,0,7,3,0.29,0.14,0,0,0,0.00,0,3,0,-3


In [5]:
print(f"Всего подач: {df_stats['Total Serves'].sum()} -> эйсы: {df_stats['Won Serves'].sum()} -> ошибки подачи: {df_stats['Lost Serves'].sum()}")

Всего подач: 24 -> эйсы: 3 -> ошибки подачи: 1


In [6]:
good_rec, all_rec = 0, 0
attacks_ponts, all_atacks = 0, 0
for index, row in df_stats.iterrows():
    all_rec += row["Total Receptions"]
    good_rec += row["Total Receptions"] * row["Good Reception Rate (R#+R+)"]

    all_atacks += row["Total Attacks"]
    attacks_ponts += row["Won Attacks"]

print(f"Прием процент: {round((good_rec / all_rec) * 100)}")
print(f"Процент успешных атак: {round((attacks_ponts/all_atacks)*100)}")
print(f"Эйсы: {df_stats['Won Serves'].sum()}")
print(f"Ошибки: {df_stats['Total Errors'].sum()}")

Прием процент: 46
Процент успешных атак: 60
Эйсы: 3
Ошибки: 9


In [30]:
df_stats.to_excel('volleyball_report.xlsx', index=False)